# NOTE

For Spark 3 Cosmos DB connector has slightly different configuration. Please select different snippest if a Spark 3.1 + pool is attached.

In [1]:
# Load a streaming Spark DataFrame from a Cosmos DB container
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

# For Spark 2.4
# dfStream = spark.readStream\
#     .format("cosmos.oltp")\
#     .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
#     .option("spark.cosmos.container", "orders")\
#     .option("spark.cosmos.changeFeed.readEnabled", "true")\
#     .option("spark.cosmos.changeFeed.startFromTheBeginning", "true")\
#     .option("spark.cosmos.changeFeed.checkpointLocation", "/localReadCheckpointFolder")\
#     .option("spark.cosmos.changeFeed.queryName", "streamQuery")\
#     .load()

# For Spark 3.1 +
dfStream = spark.readStream\
    .format("cosmos.oltp.changeFeed")\
    .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
    .option("spark.cosmos.container", "orders")\
    .option("spark.cosmos.changeFeed.startFrom", "Beginning")\
    .option("spark.cosmos.changeFeed.mode", "Incremental")\
    .load()

In [2]:
from   pyspark.sql.functions import * 
from pyspark.sql import *
from delta.tables import *

In [3]:
dfStream

In [4]:
dfStream=dfStream.withColumn("Order_Date",to_date("Order_Timestamp"))

In [5]:
dfStream=dfStream.withColumn("Order_Timestamp",to_timestamp("Order_Timestamp"))

In [6]:
dfStream

In [7]:
dfStream1=dfStream.withWatermark('Order_Timestamp', '10 minutes').groupBy("Item_Id","Order_Date",window("Order_Timestamp","60 minutes")).agg(sum("qty")).alias("sum_qty")

In [8]:
dfStream1=dfStream1.withColumnRenamed("sum(qty)","sum_qty")

In [9]:
dfStream1=dfStream1.withColumn("window_Start_Time",expr("window.start"))
dfStream1=dfStream1.withColumn("window_end_Time",expr("window.end"))

In [11]:
dfStream1.writeStream.outputMode("complete").format("delta").option("path","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/Order_stream_output").option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/tumb_checkpoint/").start()

In [12]:
# Load a streaming Spark DataFrame from a Cosmos DB container
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

# For Spark 2.4
# dfStream = spark.readStream\
#     .format("cosmos.oltp")\
#     .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
#     .option("spark.cosmos.container", "orders")\
#     .option("spark.cosmos.changeFeed.readEnabled", "true")\
#     .option("spark.cosmos.changeFeed.startFromTheBeginning", "true")\
#     .option("spark.cosmos.changeFeed.checkpointLocation", "/localReadCheckpointFolder")\
#     .option("spark.cosmos.changeFeed.queryName", "streamQuery")\
#     .load()

# For Spark 3.1 +
dfStream2 = spark.readStream\
    .format("cosmos.oltp.changeFeed")\
    .option("spark.synapse.linkedService", "CosmosDbNoSql1")\
    .option("spark.cosmos.container", "Stockitems")\
    .option("spark.cosmos.changeFeed.startFrom", "Beginning")\
    .option("spark.cosmos.changeFeed.mode", "Incremental")\
    .load()

In [13]:
dfStream1

In [14]:
dfStream2

In [15]:
dfStream2=dfStream2.withColumnRenamed("item_id","item_id_refill")

In [16]:
dfStream2=dfStream2.withColumn("refill_timestamp",to_timestamp("refill_timestamp"))

In [17]:
dfStream2=dfStream2.withColumn("refill_date",to_date("refill_timestamp"))

In [18]:
dfStream2

In [19]:
dfStream2=dfStream2.withWatermark('refill_timestamp', '10 minutes').groupBy("Item_Id_refill","refill_date",window("refill_timestamp","6 minutes")).agg(sum("stock_Refill_qty")).alias("stock_Refill_qty")

In [26]:
dfStream2=dfStream2.withColumn("stock_start_window",expr("window.start"))
dfStream2=dfStream2.withColumn("stock_end_window",expr("window.end"))
dfStream2=dfStream2.withColumnRenamed("sum(stock_Refill_qty)","sum_stock_Refill_qty")

In [27]:
dfStream2.writeStream.outputMode("complete").format("delta").option("path","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/item_Refill/").option("checkpointLocation","abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/item_refill_checkpoint/").start()

In [28]:
dfStream4=spark.readStream\
    .format("delta")\
    .option("path", "abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/item_Refill/")\
    .load()

In [29]:
dfStream5=spark.readStream\
    .format("delta")\
    .option("path", "abfss://streamsynapse@synapasestream.dfs.core.windows.net/synapse/workspaces/streamoutput/Order_stream_output/")\
    .load()

In [60]:
dfStream6=dfStream5.join(dfStream4,expr("Item_Id_refill=item_id and refill_date= Order_Date and stock_start_window= window_Start_Time and window_end_Time=stock_end_window  " ))

In [ ]:
def microbatch(batch_df,batch_id):
    url=""
    myobj=""
    df1=batch_df.withColumn("difference_quantity",expr("sum_stock_Refill_qty-sum_qty"))
    if df1.filter(df1.difference_quantity<100).count()>1:
        url = 'https://metadatamanagementemail.azurewebsites.net:443/api/sample_email/triggers/When_a_HTTP_request_is_received/invoke?api-version=2022-05-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=Lz1K7ZusrhyeY1lxp9zV12sPUJUEHxlHVH1HTwL0MMA'
        myobj = {
            "emailaddress":"rohith.amaze@gmail.com",
            "body":"triggered from python Databricks stream",
            "subject":"Low Stock Alert"
        }
        trigger_email(url,myobj)

In [61]:
def trigger_email(url,myobj):
    x = requests.post(url=url, json = myobj)

In [ ]:
dfStream6.writeStream.outputMode("append").foreachBatch(microbatch).option("checkpointLocation","/mnt/streamsynapse/synapse/workspaces/join/join_output_chkpoint/").start()